In [ ]:
#| default_exp loss

# Loss functions

> I'm lost too.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch, numpy as np, torch.nn.functional as F, torch.nn as nn

In [ ]:
#| export
def masked_mse_loss(preds, target, mask, use_mask=False, padding_mask=None):
    """
    preds:   [bs x num_patch x n_vars x patch_len]
    targets: [bs x num_patch x n_vars x patch_len] 
    mask: [bs x num_patch x n_vars]
    padding_mask: [bs x num_patch]
    """
    bs, num_patch, n_vars, patch_len = preds.shape
    loss = (preds - target) ** 2
    loss = loss.sum(dim=-1) # [bs x num_patch x n_vars]
    n_elements = torch.numel(preds)
    if padding_mask is not None: 
        loss = loss*(padding_mask==0).unsqueeze(-1) # padding mask is 0 when it is not a mask
        n_elements -= (padding_mask).sum() * patch_len * n_vars  # how many values were padded
        # this is a padding mask and should definitely not be included in calculatign the loss
    if use_mask:
        # use mask is only implemenmted for full patch masking 
        n_elements = mask.sum()
        loss = loss * mask
    loss = loss.sum() / torch.as_tensor(n_elements, device=preds.device)   # 1 number
    return loss

def masked_mae_loss(preds, target, mask, use_mask=False, padding_mask=None):
    """
    preds:   [bs x num_patch x n_vars x patch_len]
    targets: [bs x num_patch x n_vars x patch_len] 
    mask: [bs x num_patch x n_vars]
    padding_mask: [bs x num_patch]
    """
    bs, num_patch, n_vars, patch_len = preds.shape
    loss = (preds - target).abs()
    loss = loss.sum(dim=-1) # [bs x num_patch x n_vars] 
    n_elements = torch.numel(preds)
    if padding_mask is not None:
        loss = loss*(padding_mask==0).unsqueeze(-1)
        n_elements -= (padding_mask).sum() * patch_len * n_vars 
    if use_mask:
        n_elements = mask.sum()
        loss = loss * mask
    loss = loss.sum() / torch.as_tensor(n_elements, device=preds.device)  # 1 number
    return loss


def r2_score(preds, target, mask, use_mask=False):
    if use_mask:
        ss_res = (((target - preds)**2).mean(dim=-1) * mask).sum()
        ss_tot = (((target - target.mean())**2).mean(dim=-1) * mask).sum()
    else:
        ss_res = (((target - preds)**2).mean(dim=-1)).sum()
        ss_tot = (((target - target.mean())**2).mean(dim=-1)).sum()
    return 1 - (ss_res/ss_tot)

def mse(preds, target, mask, use_mask=False, padding_mask=None):
    return masked_mse_loss(preds, target, mask, use_mask=use_mask, padding_mask=padding_mask)


def rmse(preds, target, mask, use_mask=False, padding_mask=None):
    return torch.sqrt(masked_mse_loss(preds, target, mask, use_mask=use_mask, padding_mask=padding_mask))

def mae(preds, target, mask, use_mask=False, padding_mask=None):
    return masked_mae_loss(preds, target, mask, use_mask=use_mask, padding_mask=padding_mask)

def mape(preds, target, mask, use_mask=False):
    epsilon = np.finfo(np.float64).eps # from sklearn
    if use_mask:
        return (((target - preds).mean(dim=-1).abs() * mask) / (target.mean(dim=-1).abs() * mask).clamp(min=epsilon)).sum() / mask.sum()
    else:
        return (((target - preds).mean(dim=-1).abs()) / (target.mean(dim=-1).abs()).clamp(min=epsilon)).sum()
    
def cosine_similarity(preds,target, mask, use_mask=False, padding_mask=None):
    bs, num_patch, n_vars, patch_len = preds.shape
    sim = F.cosine_similarity(preds,target, dim=-1)
    n_elements = torch.numel(preds)
    if padding_mask is not None:
        sim = sim * (padding_mask==0).unsqueeze(-1)
        n_elements -= (padding_mask).sum() * patch_len * n_vars 
    if use_mask:
        n_elements = mask.sum()
        sim = sim * mask
    sim = sim.sum() / torch.as_tensor(n_elements, device=preds.device)
    return sim

def cosine_similarity_loss(preds,target, mask, use_mask=False, padding_mask=None):
    """
    preds:   [bs x num_patch x n_vars x patch_len]
    targets: [bs x num_patch x n_vars x patch_len] 
    mask: [bs x num_patch x n_vars]
    """
    bs, num_patch, n_vars, patch_len = preds.shape
    sim = F.cosine_similarity(preds,target, dim=-1)
    n_elements = torch.numel(preds)
    if padding_mask is not None:
        sim = sim * (padding_mask==0).unsqueeze(-1)
        n_elements -= (padding_mask).sum() * patch_len * n_vars
    if use_mask:
        n_elements = mask.sum()
        sim = sim * mask
    sim = sim.sum() / torch.as_tensor(n_elements, device=preds.device)
    return -sim

def huber_loss(preds, target, mask, use_mask=False, padding_mask=None, delta=1):
    """
    preds:   [bs x num_patch x n_vars x patch_len]
    targets: [bs x num_patch x n_vars x patch_len] 
    mask: [bs x num_patch x n_vars]
    padding_mask: [bs x num_patch]
    """
    bs, num_patch, n_vars, patch_len = preds.shape
    loss = F.huber_loss(preds, target, delta=delta, reduction='none')
    loss = loss.sum(dim=-1) # [bs x num_patch x n_vars] 
    n_elements = torch.numel(preds)
    if padding_mask is not None:
        loss = loss*(padding_mask==0).unsqueeze(-1)
        n_elements -= (padding_mask).sum() * patch_len * n_vars 
    if use_mask:
        n_elements = mask.sum()
        loss = loss * mask
    loss = loss.sum() / torch.as_tensor(n_elements, device=preds.device)  # 1 number
    return loss

In [ ]:
#| export
def patch_continuity_loss(preds):
    """
    preds: [bs x num_patch x n_vars x patch_len]
    targets: [bs x num_patch x n_vars x patch_len]
    """
    bs, num_patch, n_vars, patch_len = preds.shape
    # Calculate difference between end of each patch and start of next
    diff = preds[:, 1:, :, 0] - preds[:, :-1, :, -1]
    
    # Mean squared difference
    continuity_loss = torch.mean(diff**2)
    
    return continuity_loss

In [ ]:
#| notest
x = torch.randn(2,2, 2, 10)

patch_continuity_loss(x)

tensor(0.2936)

In [ ]:
#| export
class FocalLoss(nn.Module):
    """
    adapted from tsai, weighted multiclass focal loss
    https://github.com/timeseriesAI/tsai/blob/bdff96cc8c4c8ea55bc20d7cffd6a72e402f4cb2/tsai/losses.py#L116C1-L140C20
    """
    def __init__(self, 
                 weight=None, 
                 gamma=2., 
                 reduction='mean',
                 ignore_index=-100
                 ):
        super().__init__()
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction
        self.ignore_index = ignore_index
    
    __name__ = 'focalloss'
        
    def forward(self, x, y):
        """
        x: [bs x n classes x n patches]
        y: [bs x n patches]
        """
        log_prob = F.log_softmax(x, dim=1)
        prob = log_prob.exp()
        weight = self.weight.to(x.device) if self.weight is not None else None
        if y.dim() == 2:
            # hard labels
            ce = F.nll_loss(log_prob, y, weight=weight, reduction='none', ignore_index=self.ignore_index)
            loss = (1 - prob) ** self.gamma * ce.unsqueeze(1)
            mask = (y != self.ignore_index).float().unsqueeze(1)
        else:  # soft labels
            ce = -(y * log_prob)  # [bs x n_classes x n_patches]
            loss = (1 - prob) ** self.gamma * ce
            # Positions to ignore will have all zeros
            mask = (y.sum(dim=1) > 0).float().unsqueeze(1)
            
            if weight is not None:
                loss = loss * weight.view(1, -1, 1)
        if self.reduction == 'mean':
            loss = loss.sum() / mask.sum().clamp(min=1e-5)
        elif self.reduction == 'sum':
            loss = loss.sum()
        return loss

In [ ]:
#| notest
criterion = FocalLoss(gamma=0.7, weight=None, ignore_index=0)
batch_size = 10

n_patch = 721
n_class = 5
#m = torch.nn.Softmax(dim=-1)
logits = torch.randn(batch_size, n_class, n_patch)
target = torch.randint(0, n_class, size=(batch_size, n_patch))
criterion(logits, target)

tensor(8.4217)

In [ ]:
#| export
class KLDivLoss(nn.Module):
    """
    Kullback-Leibler Divergence Loss with masking for ignore_index.
    Handles soft labels with ignore_index marked as -100.
    
    Args:
        logits: [bs x n_classes x pred_labels] - model predictions
        targets: [bs x n_classes x soft_labels] - soft labels, with ignore_index positions marked as -100
        or [bs x n_labels] - hard labels
    """
    def __init__(self, weight=None, ignore_index=-100):
        super().__init__()
        self.weight = weight
        self.ignore_index = ignore_index
        self.kl_loss = nn.KLDivLoss(reduction='none')
    
    def _to_one_hot(self, targets, n_classes):
        """Convert hard labels to one-hot format"""
        valid_mask = (targets != self.ignore_index)
        # Set ignore_index positions to 0 temporarily for one-hot conversion
        valid_targets = torch.where(valid_mask, targets, torch.zeros_like(targets))
        # Convert to one-hot
        one_hot = F.one_hot(valid_targets, num_classes=n_classes).permute(0, 2, 1).float()
        # Set ignore_index positions to ignore_index
        one_hot = torch.where(valid_mask.unsqueeze(1), one_hot, 
                            torch.full_like(one_hot, self.ignore_index))
        return one_hot

    def forward(self, logits, targets):
        if targets.dim() == 2:
            targets = self._to_one_hot(targets, n_classes=logits.shape[1])
        # Create mask for valid positions (where target is not ignore_index)
        mask = (targets != self.ignore_index)  # [bs x n classes x soft_labels]
        
        # Replace ignore_index positions with zeros
        targets = torch.where(targets == self.ignore_index, 
                            torch.zeros_like(targets), 
                            targets)
        
        # Compute log probabilities
        log_probs = F.log_softmax(logits, dim=1)
        # Compute KL divergence loss
        loss = self.kl_loss(log_probs, targets)
        if self.weight is not None:
            # Reshape weight to match loss dimensions [1 x n_classes x 1]
            weight = self.weight.view(1, -1, 1).to(loss.device)
            loss = loss * weight
        # Sum across class dimension and apply mask
        loss = (loss * mask).sum() / mask.sum().clamp(min=1e-6)
        
        return loss

In [ ]:
#| notest
x = torch.randn(4,5,10)
y = torch.randint(0,5, size=(4,10))
y_og = y.clone()
y[0,0] = -100

KLDivLoss(ignore_index=-100)(x,y)


tensor(0.4065)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()